# Calculating direct emissions from all nutrient sources

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#In this cell the scenario is chosen. This can either be defined here or in the RUN_ALL.ipynb
#If defined here unmark the scen = line and select the scenario you will run.
#If defined in RUN_ALL.ipynb unmark the %store -r scen line.
#Scenario names are:
#S0_MinFert > N,P,K demand met by mineral fertilizers only
#S0_struvite_P > P demand met by struvite, N demand met by struvite and mineral fertilizer and K demand met by mineral fertilizer
#S0_compost > N,P,K supplied by compost produced in the AMB, remaining N,P,K demand met by mineral fertilizer
#S0_Ammon_salts > N supplied from recovered ammonium salts and from mineral fertilizers, P from struvite and K from mineral fertilizer
scen = 'S0_MinFert'
#%store -r scen

In [ ]:
print(scen)

In [ ]:
%store -r URBAG_map
%store -r Manure_N
%store -r Mineral_fert_N
%store -r Struvite_N
%store -r Compost_N
%store -r N_agriwaste_to_field
%store -r Mineral_fert_P2O5
%store -r Manure_P2O5
%store -r Struvite_P2O5
%store -r Ammonium_salts

In [ ]:
URBAG_map.keys()

In [ ]:
a = pd.unique(URBAG_map["Voronoi_1"])

In [ ]:
#The crop categories in this notebook are
a.sort()
a

In [ ]:
len_URBAG_map= len(URBAG_map['Voronoi_1'])
print(f"This is the amount of plots in the map in this notebook: {len_URBAG_map}")

# Emissions

In [ ]:
#Converting emissions dictionary to dataframe to fill in all emissions
d_conv_emiss_1 = pd.DataFrame(URBAG_map)
d_conv_emiss = d_conv_emiss_1.drop(['TIPO_RIEGO', 'EFIC_RIEGO', 'TIPO_DIST', 'EFIC_DISTR', 
       '01_ETO', '02_ETO', '03_ETO', '04_ETO',
       '05_ETO', '06_ETO', '07_ETO', '08_ETO', '09_ETO', '10_ETO', '11_ETO',
       '12_ETO', '01_PPT', '02_PPT', '03_PPT', '04_PPT', '05_PPT', '06_PPT',
       '07_PPT', '08_PPT', '09_PPT', '10_PPT', '11_PPT', '12_PPT',
       'kgP2O5/ha',
       'kgK2O/ha', 'kgN/ha',
       'EF_Tier2', 'id_final', 'Kc_b_Kc1', 'Kc_b_Kc2', 'Kc_b_Kc3', 'Kc_b_Kc4',
       'Kc_b_Kc5', 'Kc_b_Kc6', 'Kc_b_Kc7', 'Kc_b_Kc8', 'Kc_b_Kc9', 'Kc_b_Kc10',
       'Kc_b_Kc11', 'Kc_b_Kc12', 'Manure', 'Manure_N', 'Manure_P2O5',
       'Agriwaste_toField', 'Agriwaste_burned', 'N_agriwaste_to_field',
       "01_CWR", '02_CWR', '03_CWR', '04_CWR', '05_CWR',
       '06_CWR', '07_CWR', '08_CWR', '09_CWR', '10_CWR', '11_CWR', '12_CWR',
       '01_PPTe', '02_PPTe', '03_PPTe', '04_PPTe', '05_PPTe', '06_PPTe',
       '07_PPTe', '08_PPTe', '09_PPTe', '10_PPTe', '11_PPTe', '12_PPTe',
       '01_ID', '02_ID', '03_ID', '04_ID', '05_ID', '06_ID', '07_ID', '08_ID',
       '09_ID', '10_ID', '11_ID', '12_ID', 'LOSSES', '01_WE', '02_WE', '03_WE',
       '04_WE', '05_WE', '06_WE', '07_WE', '08_WE', '09_WE', '10_WE', '11_WE',
       '12_WE', 'WE_m3_ha_yr',], axis=1)
d_conv_emiss.head()

In [ ]:
d_conv_emiss.keys()

## Emission factors definition

In [ ]:
#Emission factors for NH3,NOx, NO3-, N2O Direct, N2O Indirect Volatiliaziton, N2O Indirect leaching, PO43-
#For N2O Direct, it is the reduction % applied to the Tier 2 EFs from Mendoza Beltran et.al (2022) https://doi.org/10.1016/j.scitotenv.2022.153514
EFs = pd.read_excel("inputs/EFs_20230210.xlsx")

In [ ]:
EFs.keys()

In [ ]:
EFs["NH3"].loc[EFs["source"]=="compost"]

In [ ]:
#Units conversion factors
NH3_N_to_NH3 = 17/14
NH3_to_NH3_N = 14/17
N2O_N_to_N2O = 44/28
N2O_to_N2O_N = 28/44
NO2_N_to_NO2 = 46/14
NO2_to_NO2_N = 14/46
NO3_N_to_NO3 = 62/14
NO3_to_NO3_N = 14/62
C_to_CO2 = 44/12
P_to_P2O5 = 2.29
P2O5_to_P = 0.44

In [ ]:
cons23 = np.repeat((2.5/0.466), len_URBAG_map) #IPCC 2019 chapter 2 table 2.5 Emission factor 2.5 g NOx / kg DM Burnt
cons24 = np.repeat((2.7/0.466), len_URBAG_map) #IPCC 2019 chapter 2 table 2.5 Emission factor 2.7 g CH4 / kg DM Burnt
cons25 = np.repeat((0/0.466), len_URBAG_map) #IPCC 2019 chapter 2 table 2.5 Emission factor 1515 g CO2 / kg DM Burnt
cons26 = np.repeat((92/0.466), len_URBAG_map) #IPCC 2019 chapter 2 table 2.5 Emission factor 92 g CO / kg DM Burnt
cons27 = np.repeat((0.07/0.466), len_URBAG_map) #IPCC 2019 chapter 2 table 2.5 Emission factor 0.07 g N2O / kg DM Burnt
cons28 = np.repeat((0), len_URBAG_map) #No emissions of nitrate and phosphate for struvite use

conv1 = np.repeat(NH3_N_to_NH3, len_URBAG_map) #Conversion from NH3 as N to NH3
conv2 = np.repeat(N2O_N_to_N2O, len_URBAG_map) #Conversion from N2O-N to N2O
conv3 = np.repeat(NO2_N_to_NO2, len_URBAG_map) #Conversion from kg NOx-N to kg NOx-NO2 as used in ecoinvent
conv4 = np.repeat(NO3_N_to_NO3, len_URBAG_map) #Conversion from NO3-N to NO3
conv5 = np.repeat(C_to_CO2, len_URBAG_map) #Conversion from C to CO2
conv6 = np.repeat(NH3_to_NH3_N, len_URBAG_map) #Conversion from NH3 to NH3-N
conv7 = np.repeat(NO2_to_NO2_N, len_URBAG_map) #Conversion NO2 to NO2-N
conv9 = np.repeat(P_to_P2O5, len_URBAG_map) #Conversion P to P2O5
conv10 = np.repeat(P2O5_to_P, len_URBAG_map) #Conversion P2O5 to P

Inventoried emissions:

- Emissions to Air:
    - Ammonia (NH3) [kg NH3/ha]
        - From solid manure > Ecoinvent
        - From mineral fertilisers > Ecoinvent
        - Struvite > No ammonia release according to Wang et al (2023) 
        - Compost > Bn et.al(2006)
    - Nitrogen oxides (NOx) [kg NO2/ha]
        - From mineral and organic fertiliser and excreta applied > WFLDB guidelines pg 38, that uses emission factors from EEA (2013).
        - Burned Agri Waste > IPCC (2019)
        - Struvite > assumed as if it was mineral fertilizers
    - Methane (CH4) [kg CH4/ha]
        - Burned Agri Waste > IPCC (2019)
    - Carbon dioxide (CO2) [kg CO2/ha]
        - From Urea > IPCC (2006) as no update in IPCC (2019)
        - Burned Agri Waste > IPCC (2019)
    - Carbon monoxide (CO) [kg CO/ha]
        - Burned Agri Waste > IPCC (2019)
    - Nitrous oxides (N2O) [kg N2O/ha]
        - Direct N2O Manure and mineral fertilizers together > IPCC (2019) Tier 2 - Using EFs from crop level meta-analysis (Mendoza Beltran et al 2022)
        - Direct N2O from Compost > Brunn et.al(2006)
        - Indirect N2O > IPCC (2019) Tier 1
        - Burned Agri Waste > IPCC (2019) Tier 1 - From above and below ground biomass
        - Struvite > Reduced Tier 2 emissions factor by 0.592 which is the mitigation found in Wang et.al (2023)

- Emissions to water:
    - Phosphate (PO43-) [kg PO43-/ha]
        - run-Off to Surface Waters manure and mineral fertilizer application > Ecoinvent P-SALCA model
        - Struvite >  No phosphate leaching according to Wang et al (2023)
        - Compost >  No phosphate leaching assumed not included in Brunn et.al (2006)
    - Phosphorus (P) [kg P/ha]
        - Erosion of soil > Ecoinvent > Not implemented!!(No map of soil erosion)

- Emissions to ground water:
    - Phosphate (PO43-) [kg PO43-/ha]
        - Leaching to ground water > Ecoinvent (hold only for slurry and sewage sludge application)
    - Nitrate (NO3-) [kg NO3-/ha]
        - Leaching to ground water manure and mineral fertilizer application > IPCC (2019)
        - Struvite >  No nitrate leaching according to Wang et al (2023)
        - Compost >  No nitrate leaching assumed not included in Brunn et.al (2006)
        
        
* PEFCR > https://eplca.jrc.ec.europa.eu/permalink/PEFCR_guidance_v6.3-2.pdf
* ecoinvent > https://db.ecoinvent.org/reports/15_Agriculture.pdf
* IPCC (2019) > https://www.ipcc-nggip.iges.or.jp/public/2019rf/pdf/4_Volume4/19R_V4_Ch11_Soils_N2O_CO2.pdf
* IPCC (2006) > https://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/4_Volume4/V4_11_Ch11_N2O&CO2.pdf
* Mendoza Beltran et al (2022) > https://www.sciencedirect.com/science/article/pii/S0048969722006064?via%3Dihub
* WFLDB >  https://simapro.com/wp-content/uploads/2020/11/WFLDB_MethodologicalGuidelines_v3.5.pdf
* wang et.al(2023) > https://doi.org/10.1016/j.jenvman.2022.117143
* Brunn et.al (2006) > 

## Emissions from burning agricultural waste onsite

### NOx, CH4, CO2, CO, N2O

For now these are zero as we do not consider agricultural residues

In [ ]:
agriwaste_burned = URBAG_map['Agriwaste_burned']

In [ ]:
#IPCC 2019 chapter 2 table 2.5 Emission factor 2.5 g NOx / kg DM Burnt
d_conv_emiss['NOx_burnwaste_air'] = [(x * y)/1000 for x, y in zip(agriwaste_burned, cons23)] # kg NOx/ha

In [ ]:
#IPCC 2019 chapter 2 table 2.5 Emission factor 2.7 g CH4 / kg DM Burnt
d_conv_emiss['CH4_burnwaste_air'] = [(x * y)/1000 for x, y in zip(agriwaste_burned, cons24)] # kg CH4/ha

In [ ]:
#IPCC 2019 chapter 2 table 2.5 Emission factor 1515 g CO2 / kg DM Burnt
##Quote from table 2.5 IPCC(2019)!!!!!
#"Note: For combustion of non-woody biomass in Grassland and Cropland, 
#CO2 emissions do not need to be estimated and reported, because it is assumed that annual CO2 removals (through growth)
#and emissions (whether by decay or fire) by biomass are in balance (see earlier discussion on synchrony in Section 2.4."
d_conv_emiss['CO2_burnwaste_air'] = [(x * y)/1000 for x, y in zip(agriwaste_burned, cons25)] # kg CO2/ha

In [ ]:
#IPCC 2019 chapter 2 table 2.5 Emission factor 92 g CO / kg DM Burnt
d_conv_emiss['CO_burnwaste_air'] = [(x * y)/1000 for x, y in zip(agriwaste_burned, cons26)] # kg CO/ha

In [ ]:
#IPCC 2019 chapter 2 table 2.5 Emission factor 0.07 g N2O / kg DM Burnt
d_conv_emiss['N2O_burnwaste_air'] = [(x * y)/1000 for x, y in zip(agriwaste_burned, cons27)] # kg N2O/ha

## Ammonia (NH3) to Air

#### MANURE

SOURCE>> PEFCR pag 74.

    NH3_manure_air [Kg NH3/ha] = (EF_Manure_NH3_air * M * TAN)

- EF_Manure_NH3_air = 0.24 kg NH3/kg N manure applied
- M = Manure applied (tons/ha)
- TN_catle = Total N in manure - solid (cattle and pigs) kg N/ton

In [ ]:
# kg N/ha * kg NH3/kg N = kg NH3/ha
d_conv_emiss['NH3_manure_air'] = Manure_N*EFs["NH3"].loc[EFs["source"]=="manure"].values
d_conv_emiss['NH3_manure_air']

#### MINERAL FERTILIZER AND RECOVERED AMMONIUM SALTS

SOURCE>> Ecoinvent pg 29.

    NH3_fert_air [kg NH3/ha] = Sum of (EF_fert * Fert_applied) * NH3_N_to_NH3
    
- NH3fert = NH3 emissions from mineral fertilisers
- EF_fert = Emission factor per fertiliser type (kg NH3-N/kg N applied)
    - Ammonium nitrate : 2%
    - Urea: 15%
    - Others: 4%
- Fert_applied = Applied mineral fertilisers (kg N/ha)

#ALTERNATIVE FROM WFLDB >> Emission factors from EEA 2013 3.D Table 3-2, per kg N of the mineral fertilizer in basic soil ph >7: 

#For Ammonium nitrate is 0.03 and for Urea is 0.2. 

#As very similar to ecoinvent we left the ecoinvent values. 

In [ ]:
#kg N/ha * kg NH3-N/kg N * NH3-N_to_NH3 = kg NH3/ha
d_conv_emiss['NH3_fert_air'] = (Ammonium_salts+Mineral_fert_N)*EFs["NH3"].loc[EFs["source"]=="mineral_fert"].values*conv1
d_conv_emiss['NH3_fert_air']

#### STRUVITE


In [ ]:
#No leaching Wang et.al (2023), thus zero NH3 emissions
#kg N/ha * kg NH3-N/kg N * NH3-N_to_NH3 = kg NH3/ha
d_conv_emiss['NH3_struv_air'] = Struvite_N*EFs["NH3"].loc[EFs["source"]=="struvite"].values*conv1
d_conv_emiss['NH3_struv_air']

#### COMPOST

In [ ]:
#No leaching Wang et.al (2023), thus zero NH3 emissions
#kg N/ha * kg NH3-N/kg N * NH3-N_to_NH3 = kg NH3/ha
d_conv_emiss['NH3_compost_air'] = Compost_N*EFs["NH3"].loc[EFs["source"]=="compost"].values*conv1
d_conv_emiss['NH3_compost_air']

#### TOTAL Ammonia (NH3) TO AIR

    NH3_total_air [kg NH3/ha] = NH3_manure_air + NH3_fert_air + NH3_agri_waste_air + NH3_struv_air + NH3_compost_air

In [ ]:
d_conv_emiss['NH3_total_air'] = d_conv_emiss['NH3_fert_air'] + d_conv_emiss['NH3_manure_air'] + d_conv_emiss['NH3_struv_air'] + d_conv_emiss['NH3_compost_air']
d_conv_emiss['NH3_total_air'] 

## Nitrogen Oxides (NOx) to Air

#### TOTAL APPLIED N - Fertilizers, recovered ammonium salts, manure, excreta

SOURCE>> WFLDB guidelines pg 38, that uses emission factors from EEA (2013).

    NOx_NInputs [kg NO2/ha] = EFfert_man * (Manure + Fert) * NO2_N_to_NO2

- NOx_appliedN = Nitrogen oxides emissions from nitrification process
- Manure = Applied solid manure (kg N/ha)
- Fert = Applied mineral fertilisers (kg N/ha)
- EFfert_man =  Emission factor for the application of mineral and organic fertilisers (kg NOx-N/kg N applied), value: 0.012 (EEA, 2013, 3.D Tab 3-1)

In [ ]:
#N inputs is manure + mineral
# kg N/ha * kg NO2-N/kg N applied * kg NO2/kg NO2_N = kg NO2/ha
d_conv_emiss['NOx_manure_fert_air'] = (Ammonium_salts+Mineral_fert_N + Manure_N) * EFs["NOx"].loc[EFs["source"]=="mineral_fert"].values * conv3
d_conv_emiss['NOx_manure_fert_air']

#### STRUVITE USE

Same as for total N applied - from WFLDB 0.012 kg NOx-N/kg N applied, given no better evidence so far

In [ ]:
# kg N/ha * kg NO2-N/kg N applied * kg NO2/kg NO2_N = kg NO2/ha
d_conv_emiss['NOx_struv_air'] = Struvite_N * EFs["NOx"].loc[EFs["source"]=="struvite"].values * conv3
d_conv_emiss['NOx_struv_air']

#### COMPOST USE

No NOx emissions included

#### Total NOx emissions to air

    NOx Total [kg NOx/ha] = NOx from manure and fert N inputs + NOx from agricultural waste burned onsite + NOx from struvite use

In [ ]:
d_conv_emiss['NOx_total_air'] = d_conv_emiss['NOx_manure_fert_air'] + d_conv_emiss['NOx_burnwaste_air'] + d_conv_emiss['NOx_struv_air']
d_conv_emiss['NOx_total_air'] 

## Nitrate (NO3-) to river emissions

#### Mineral Fert, recovered ammonium salts, Manure and Residues

IPCC (2019) 

    N03_leach_runoff [kg NO3/ha yr]= [Frac_Leach * (Fert_N + Manure_N + Residues_N + Mineralised_N)] * NO3_N_to_NO3
    
- N03_leach_runoff = fraction of all N added to/mineralised in managed soils in regions where leaching/runoff occurs that is lost through leaching and runoff (kg N/kg of N additions) (Table 11.3)
- Fert_N = Amount of mineral fertilizer (kg N/ha)
- Manure_N = Amount of organic fertilizer (kg N/ha)
- Residues_N = Amount of nitrogen from residual crop (kg N/ha)
- Mineralised_N = Amount of nitrogen from mineralization (kg N/ha) #Not included 
- Frac_Leach = fraction of all N added to/mineralised in managed soils in regions where leaching/runoff
occurs that is lost through leaching and runoff, kg N (kg of N additions)-1 (Table 11.3) = 0.2)

In [ ]:
#kg N/ha * kg NO3-N/kg N * NO3-3_to_NO3 = kg NO3/ha
d_conv_emiss['NO3_leaching_runoff'] = (Ammonium_salts+Mineral_fert_N + Manure_N + N_agriwaste_to_field) * EFs["NO3-"].loc[EFs["source"]=="mineral_fert"].values * conv4
d_conv_emiss['NO3_leaching_runoff'] 

#### Struvite

No leaching to groundwater  or runoff to rivers from struvite use is assumed.

Slow realease of struvite (low soluility) decreases P leached and enables a better absorption by the plant. 

See Arcas-Pilz et.al (2021) > https://doi.org/10.1016/j.scitotenv.2021.149424 hydroponic

Wang et.al (2023) > https://doi.org/10.1016/j.jenvman.2022.117143 open air cultivation

In [ ]:
#kg N/ha * kg NO3-N/kg N * NO3-3_to_NO3 = kg NO3/ha
d_conv_emiss['NO3_leaching_runoff_struv'] = Struvite_N * EFs["NO3-"].loc[EFs["source"]=="struvite"].values * conv4
d_conv_emiss['NO3_leaching_runoff_struv']

#### COMPOST

Leaching of nitrate from Brunn et al (2006)

In [ ]:
#kg N/ha * kg NO3-N/kg N * NO3-3_to_NO3 = kg NO3/ha
d_conv_emiss['NO3_leaching_runoff_compost'] = Compost_N * EFs["NO3-"].loc[EFs["source"]=="compost"].values * conv4
d_conv_emiss['NO3_leaching_runoff_compost']

#### Total NO3 leaching and runoff

In [ ]:
d_conv_emiss['NO3_total'] = d_conv_emiss['NO3_leaching_runoff'] + d_conv_emiss['NO3_leaching_runoff_struv'] + d_conv_emiss['NO3_leaching_runoff_compost']
d_conv_emiss['NO3_total']

## Nitrous oxide (N2O) to Air

#### DIRECT N2O FROM TOTAL N INPUTS - Mineral Fert, Manure, Residues
SOURCE> IPCC (2019) Tier 2 Equation for direct emissions with specific emission factors for different crops where available and where not IPCC default EF

    N2ODirect [kg N2O/ha]  = [EFs_crop_level * (Fert_N + Manure_N) + (Residues_N + Mineralised_N) * EF_IPCC] * N2O_N_to_N2O
    
- EFs_crop_level = Emission factors for individual crops determined by a meta-analysis based on Cayuela et.al. (kg N2O-N/kg applied N) See Kelzy's TFM.
- EF_IPCC = 0.01 kg N2O-N/kg N applied
- Fert_N = Amount of mineral fertilizer (kg N/ha)
- Manure_N = Amount of organic fertilizer (kg N/ha)
- Residues_N = Amount of nitrogen from residual crop (kg N/ha)
- Mineralised_N = Amount of nitrogen from mineralization (kg N/ha) #Not included 

#### Residues

In [ ]:
#kg N/ha * kg N2O-N/kg N * N2O-N_to_N2O = kg N2O/ha
N_residues_emiss = N_agriwaste_to_field * EFs["N2O Direct"].loc[EFs["source"]=="residues"].values * conv2
N_residues_emiss

#### Mineral fertilizer, recovered ammonium salts, manure

In [ ]:
#kg N/ha * kg N2O-N/kg N * N2O-N_to_N2O = kg N2O/ha
N_input_emiss = URBAG_map['EF_Tier2'] * (Ammonium_salts+Mineral_fert_N + Manure_N) * conv2
N_input_emiss

#### Mineral fertilizer, recovered ammonium salts, manure, residues

In [ ]:
#kg N2O/ha
d_conv_emiss['N2O_direct_air'] = N_residues_emiss + N_input_emiss
d_conv_emiss['N2O_direct_air']

#### STRUVITE

We use the 60% of Tier 2 EFs for N from struvite, according to Wang et.al(2023) mitigation of N2O emissions from struvite use

In [ ]:
#kg N/ha * kg N2O-N/kg N * emission reduction% * N2O-N_to_N2O = kg N2O/ha
d_conv_emiss['N2O_direct_air_struv'] = Struvite_N * URBAG_map['EF_Tier2'] * EFs["N2O Direct"].loc[EFs["source"]=="struvite"].values * conv2
d_conv_emiss['N2O_direct_air_struv']

#### COMPOST

Based on Brunn et al (2006)

In [ ]:
#kg N/ha * kg N2O-N/kg N * N2O-N_to_N2O = kg N2O/ha
d_conv_emiss['N2O_direct_air_compost'] = Compost_N * EFs["N2O Direct"].loc[EFs["source"]=="compost"].values * conv2
d_conv_emiss['N2O_direct_air_compost']

#### Total N2O Direct emissions

In [ ]:
#kg N2O/ha
d_conv_emiss['N2O_direct_air_total'] = d_conv_emiss['N2O_direct_air'] + d_conv_emiss['N2O_direct_air_struv'] + d_conv_emiss['N2O_direct_air_compost']
d_conv_emiss['N2O_direct_air_total']

#### INDIRECT N2O emissions from N volatilized - Mineral ferti, manure, struvite, compost

SOURCE> IPCC (2019)     

    N2OIndirect_volatilized_N [kg N2O/ha] = (NH3tot + NOx) * EFN2ODepo * NO2_N_to_NO2
   
- N2OIndirect_volatilized_N = annual amount of N2O produced from atmospheric deposition of N volatilised from managed soils
- NH3Tot = total N volatilised as NH3 = Total NH3 emissions to air (kg NH3-N/ha)
- NOx = Total N volatilised as NOx = NOx emissions to air from total N inputs (kg NOx-N/ha)
- EFN2ODepo = Emission factor for N2O emissions from atmospheric depositions of N on soil and water surfaces (kg N2O-N / kg NH3-N + NOx-N volaitised) = 0.01

In [ ]:
NH3tot_aux = d_conv_emiss['NH3_total_air']
NH3Volat = NH3tot_aux * conv6 #NH3 to NH3-N
NOx_aux = d_conv_emiss['NOx_total_air']
NOxVolat = NOx_aux * conv7 #from NOx-NO2 to NOx-N
NH3_NOx_Volat = NH3Volat + NOxVolat #kg NH3-N/ha + NOx-N/ha 
NH3_NOx_Volat_aux = NH3_NOx_Volat * EFs["N2O Indirect Volatiliaziton"].loc[EFs["source"]=="Volatilized_NH3_Nox"].values
d_conv_emiss['N2O_indirect_Volat_air'] = NH3_NOx_Volat_aux * conv2 #from kg N2O-N to kg N2O
d_conv_emiss['N2O_indirect_Volat_air']

#### INDIRECT N2O emissions from N leaching and run off (Mineral Fert, Manure, Struvite, compost)
SOURCE> IPCC (2019)     

    N2OIndirect_leach_runoff_N [kg N2O/ha] = (NO3_leaching) * EFN2OLeachrunOff * NO2_N_to_NO2
   
- N2OIndirect_leach_runoff_N = annual amount of N2O produced from N leaching and run off from N additions to managed soils in regions where leaching/runoff occurs
- NO3_leaching = total N leaching and run off (kg NO3-N/ha)
- EFN2OLeachrunOff = Emission factor for N2O emissions from N leaching and run off (kg N2O-N / kg N leached and runoff) = 0.011

In [ ]:
N2OIndirect_leach_runoff_N_aux = d_conv_emiss['NO3_total']
N2OIndirect_leach_runoff_N = N2OIndirect_leach_runoff_N_aux * EFs["N2O Indirect leaching"].loc[EFs["source"]=="Leached_NO3"].values
d_conv_emiss['N2O_indirect_Leachrunoff_air'] = N2OIndirect_leach_runoff_N * conv2 # kg N2O/ha yr
d_conv_emiss['N2O_indirect_Leachrunoff_air']

#### TOTAL INDIRECT N2O emissions

SOURCE > IPCC (2019)

    N2O_Indirect [kg N2O/ha yr] = N2OIndirect_volatilized_N + N2OIndirect_leach_runoff_N



In [ ]:
d_conv_emiss['N2O_indirect_air'] = d_conv_emiss['N2O_indirect_Volat_air'] + d_conv_emiss['N2O_indirect_Leachrunoff_air'] #kg N2O/ha

#### TOTAL N2O emissions to air

    N2OTot [kg N2O/ha] =  N2ODirect + N2OIndirect + N2OBurn_Waste

In [ ]:
d_conv_emiss['N2O_total_air'] = d_conv_emiss['N2O_direct_air_total'] + d_conv_emiss['N2O_indirect_air'] + d_conv_emiss['N2O_burnwaste_air']

## Carbon dioxide (CO2) to Air

#### UREA APPLICATION

SOURCE>> IPCC(2006) 

    CO2urea [kg CO2/ha] = EFurea * UreaApp * C_to_CO2
    
- EFurea = Apply an overall emission factor (EF) of 0.20 for urea, which is equivalent to the carbon content of urea on an atomic weight basis (20% for CO(NH2)2).
- UreaApp = Applied urea (kg N/ha)

In [ ]:
#No data on urea use
d_conv_emiss['CO2_urea_air'] = 0 
d_conv_emiss['CO2_urea_air']

#### Total CO2 emissions to air

    CO2 Total [kg CO2/ha] = CO2 from urea + CO2 from agricultural waste burned onsite

In [ ]:
d_conv_emiss['CO2_total_air'] = d_conv_emiss['CO2_urea_air'] + d_conv_emiss['CO2_burnwaste_air']
d_conv_emiss['CO2_total_air']

## Phosphate (PO43-) to Water

#### Manure and mineral fertilizers

SOURCE>> Ecoinvent pg 35.

    PrunOff [kg/ha*a] = Prol * Fro
    Fro = 1 + (0.2/80 * fert) + (0.7/80 * Slurry/liquid sewage sludge) + (0.4/80 * Manure)
    
- PrunOff = Lost P through run-off to rivers
- Prol = Average quantity of P lost through run-off for a land use category in (kg/ha*a), here value: 0,175 kg P/ha * a for open arable land
- Fro = correction factor for fertilisation with P
- Fert = Mineral fertiliser applied (kg P2O5/ha)
- Slurry/liquid sewage sludge = Applied (kg P2O5/ha)
- Manure = Applied solid manure (kg P2O5/ha) = Manure (ton DM/ha) * TP2O5_sol_cattle_pigs 

In [ ]:
PO43_aux1 = Mineral_fert_P2O5 * EFs["PO43-"].loc[EFs["source"]=="mineral_fert"].values
PO43_aux2 = Manure_P2O5 * EFs["PO43-"].loc[EFs["source"]=="manure"].values
Fro = 1 + PO43_aux1 + PO43_aux2
d_conv_emiss['PO43_runoff_water'] = np.repeat(0.175, len(Fro)) * Fro #kg P/ha yr
d_conv_emiss['PO43_runoff_water']

In [ ]:
PO43_runoff_emiss = d_conv_emiss['PO43_runoff_water'] #kg P/ha yr

In [ ]:
#Condition 1: Plots where inputs are larger than 0 emissions should be what estimated by ecoinvent equation
#Condition 2: Plots where inputs are 0 emissions should be 0
cond1 = PO43_aux1 + PO43_aux2 > 0
cond2 = PO43_aux1 + PO43_aux2 <= 0

PO43_runoff_emiss.loc[PO43_runoff_emiss.index[cond1]] = PO43_runoff_emiss
PO43_runoff_emiss.loc[PO43_runoff_emiss.index[cond2]] = 0

In [ ]:
d_conv_emiss['PO43_runoff_water'] = PO43_runoff_emiss # kg P/ha yr
pd.unique(d_conv_emiss.loc[d_conv_emiss['PO43_runoff_water']==0]["Voronoi_1"])

#### Struvite

No run-off to surface waters from struvite use is assumed.

Slow realease of struvite (low soluility) decreases P leached and enables a better absorption by the plant. 

See Arcas-Pilz et.al (2021) > https://doi.org/10.1016/j.scitotenv.2021.149424 hydroponic

Wang et.al (2023) > https://doi.org/10.1016/j.jenvman.2022.117143 open air cultivation

In [ ]:
PO43_runoff_emiss_struv = Struvite_P2O5 * EFs["PO43-"].loc[EFs["source"]=="struvite"].values
d_conv_emiss['PO43_runoff_struv'] = PO43_runoff_emiss_struv # kg P/ha yr
d_conv_emiss['PO43_runoff_struv']

#### Compost

No run-off to surface waters from compost is assumed.

#### Total PO43- emissions

In [ ]:
#kg P/ha yr
d_conv_emiss['PO43_runoff_water_total'] = d_conv_emiss['PO43_runoff_water'] + d_conv_emiss['PO43_runoff_struv']

## Phosphorus to water

#### EROSION BY WATER

SOURCE>> ecoinvent pg 35 

    Per [kg P/ha * a] = 10000 * Ser * Pcs _Fr * Ferw

- Per = Quantity of P emitted through erosion to rivers
- Ser = Quantitiy of soil eroded (kg/ha*a)
- Pcs = P content in top soil kg P/kg soil. Average value used 0.00095 kg/kg
- Fr = enrichment factor for P. Average value of 1.86
- Ferw = fraction of the eroded soil that reaches the river. Average value of 0.2

In [ ]:
#To be implemented?? If data for Quantity of soil eroded becomes available

## Phosphate (PO43-) to Groundwater

#### PHOSPHATE LEACHING TO GROUNDWATER

SOURCE>> ecoinvent pg 34.

NOTE>> Only for slurry and sewage sludge

    Pleach [kg/ha * a] = Pgwl * Fgw
    Fgw = 1 + (0.2/80 * (Slurry or liquid sewage sludge P2O5 in kg/ha))

- Pleach = Phosphorus leaching to ground water
- Fgw = correction factor for fertilisation by slurry
- Pgwl = Average quantity of P leached to ground water for a land use category (kg/ha * a), here value: 0,07 kg P/ha * a for arable land

In [ ]:
#Implement if data on slurry and sewage application becomes available

#### Compost

No leaching to ground waters from compost is assumed.

### Converting emissions to float

In [ ]:
d_conv_emiss['NH3_manure_air']               = pd.to_numeric(d_conv_emiss['NH3_manure_air'], downcast='float')
d_conv_emiss['NH3_fert_air']                 = pd.to_numeric(d_conv_emiss['NH3_fert_air'], downcast='float')
d_conv_emiss['NH3_struv_air']                = pd.to_numeric(d_conv_emiss['NH3_struv_air'], downcast='float')
d_conv_emiss['NH3_compost_air']              = pd.to_numeric(d_conv_emiss['NH3_compost_air'], downcast='float')
d_conv_emiss['NH3_total_air']                = pd.to_numeric(d_conv_emiss['NH3_total_air'], downcast='float')
d_conv_emiss['NOx_manure_fert_air']          = pd.to_numeric(d_conv_emiss['NOx_manure_fert_air'], downcast='float')
d_conv_emiss['NOx_burnwaste_air']            = pd.to_numeric(d_conv_emiss['NOx_burnwaste_air'], downcast='float')
d_conv_emiss['NOx_struv_air']                = pd.to_numeric(d_conv_emiss['NOx_struv_air'], downcast='float')
d_conv_emiss['NOx_total_air']                = pd.to_numeric(d_conv_emiss['NOx_total_air'], downcast='float')
d_conv_emiss['CO2_urea_air']                 = pd.to_numeric(d_conv_emiss['CO2_urea_air'], downcast='float')
d_conv_emiss['CO2_burnwaste_air']            = pd.to_numeric(d_conv_emiss['CO2_burnwaste_air'], downcast='float')
d_conv_emiss['CO2_total_air']                = pd.to_numeric(d_conv_emiss['CO2_total_air'], downcast='float')
d_conv_emiss['CO_burnwaste_air']             = pd.to_numeric(d_conv_emiss['CO_burnwaste_air'], downcast='float')
d_conv_emiss['CH4_burnwaste_air']            = pd.to_numeric(d_conv_emiss['CH4_burnwaste_air'], downcast='float')
d_conv_emiss['PO43_runoff_water']            = pd.to_numeric(d_conv_emiss['PO43_runoff_water'], downcast='float')
d_conv_emiss['PO43_runoff_struv']            = pd.to_numeric(d_conv_emiss['PO43_runoff_struv'], downcast='float')
d_conv_emiss['PO43_runoff_water_total']      = pd.to_numeric(d_conv_emiss['PO43_runoff_water_total'], downcast='float')
d_conv_emiss['NO3_leaching_runoff']          = pd.to_numeric(d_conv_emiss['NO3_leaching_runoff'], downcast='float')
d_conv_emiss['NO3_leaching_runoff_struv']    = pd.to_numeric(d_conv_emiss['NO3_leaching_runoff_struv'], downcast='float')
d_conv_emiss['NO3_leaching_runoff_compost']  = pd.to_numeric(d_conv_emiss['NO3_leaching_runoff_compost'], downcast='float')
d_conv_emiss['NO3_total']                    = pd.to_numeric(d_conv_emiss['NO3_total'], downcast='float')
d_conv_emiss['N2O_direct_air']               = pd.to_numeric(d_conv_emiss['N2O_direct_air'], downcast='float')
d_conv_emiss['N2O_direct_air_struv']         = pd.to_numeric(d_conv_emiss['N2O_direct_air_struv'], downcast='float')
d_conv_emiss['N2O_direct_air_compost']       = pd.to_numeric(d_conv_emiss['N2O_direct_air_compost'], downcast='float')
d_conv_emiss['N2O_direct_air_total']         = pd.to_numeric(d_conv_emiss['N2O_direct_air_total'], downcast='float')
d_conv_emiss['N2O_indirect_Volat_air']       = pd.to_numeric(d_conv_emiss['N2O_indirect_Volat_air'], downcast='float')
d_conv_emiss['N2O_indirect_Leachrunoff_air'] = pd.to_numeric(d_conv_emiss['N2O_indirect_Leachrunoff_air'], downcast='float')
d_conv_emiss['N2O_indirect_air']             = pd.to_numeric(d_conv_emiss['N2O_indirect_air'], downcast='float')
d_conv_emiss['N2O_burnwaste_air']            = pd.to_numeric(d_conv_emiss['N2O_burnwaste_air'], downcast='float')
d_conv_emiss['N2O_total_air']                = pd.to_numeric(d_conv_emiss['N2O_total_air'], downcast='float')

In [ ]:
test = d_conv_emiss['N2O_total_air']
type(test[0])

In [ ]:
#Total emissions before balance in kg N per year
N2O_total_before_balance = np.sum(d_conv_emiss['Area']*(1/10000) * d_conv_emiss['N2O_total_air'])*N2O_to_N2O_N/1000
NH3_total_before_balance = np.sum(d_conv_emiss['Area']*(1/10000) * d_conv_emiss['NH3_total_air'])*NH3_to_NH3_N/1000
NOx_total_before_balance = np.sum(d_conv_emiss['Area']*(1/10000) * d_conv_emiss['NOx_total_air'])*NO2_to_NO2_N/1000
NO3_total_before_balance = np.sum(d_conv_emiss['Area']*(1/10000) * d_conv_emiss['NO3_total'])*NO3_to_NO3_N/1000
PO43_total_before_balance = np.sum(d_conv_emiss['Area']*(1/10000) * d_conv_emiss['PO43_runoff_water_total'])*P_to_P2O5/1000

In [ ]:
Emiss_totals_before_balances = np.array([[N2O_total_before_balance,
                     NH3_total_before_balance,
                     NOx_total_before_balance,
                     NO3_total_before_balance,
                     PO43_total_before_balance]])

In [ ]:
Totals_Emissions = pd.DataFrame(Emiss_totals_before_balances, columns = ['N2O_NtonYR',
                                               'NH3_NtonYR',
                                               'NOx_NtonYR',
                                               'NO3-_NtonYR',
                                               'PO43-_P2O5tonYR'])

In [ ]:
#3.041373
#Total before balance (after balance they should not change) ton N/yr 
Totals_Emissions_AMB = Totals_Emissions.T
Totals_Emissions_AMB

## Test for municipalities

In [ ]:
pd.unique(d_conv_emiss['Municipi'])

In [ ]:
test = d_conv_emiss.loc[d_conv_emiss['Municipi']=='Badalona']
test.loc[test['CAT_NIV_5']=='Horta familiar']['PO43_runoff_water_total']

In [ ]:
URBAG_map.keys()

## Saving outputs

In [ ]:
%store d_conv_emiss
%store URBAG_map

In [ ]:
#Emissions in kg compound/ha
emissions = pd.DataFrame(d_conv_emiss)
emissions.to_csv("output/" + scen + "/emissions/LET_emissions_before_bln.csv")

## Vizualization

In [ ]:
d_conv_emiss.keys().values

In [ ]:
emiss_boxplot = d_conv_emiss.drop(["DUN_GRUP", 'DUN_CULTIU', 'Voronoi_1', 'TF', 'REG_CREAF',
       'REG_CAT', 'TF_REG', 'CAT_NIV_5', 'ORD_NIV_5', 'CAT_NIV_4',
       'ORD_NIV_4', 'CAT_NIV_3', 'ORD_NIV_3', 'CAT_NIV_2', 'ORD_NIV_2',
       'CAT_NIV_1F', 'ORD_NIV_1F', 'CAT_NIV_1', 'ORD_NIV_1', 'ID_Voronoi',
       'Municipi', 'Codi_munic', 'Area', 'Fid', 'Comarca', 'Codi_comar',
       'ID_CUENCA', 'CUENCA', 'ID_SUBCUEN', 'N_SUBCUENC',
       'SUBCUENCAS', 'yield_kgHA', 'N_T', 'P_T', 'K_T', 'Voronoi_ag',
       'Vor_ag_Num', 'FoodTons_N',], axis=1)

In [ ]:
sns.boxplot(data=emiss_boxplot, orient="h", showfliers=False)

In [ ]:
emiss_boxplot1 = []
for x in emiss_boxplot.keys():
    print(x)
    emiss_boxplot1.append(emiss_boxplot[x])

In [ ]:
fig, axs = plt.subplots(10, 3, figsize=(10,20))
axs[0, 0].boxplot(emiss_boxplot1[0], 0, '', labels = [''], showmeans=True)
axs[0, 0].set_title("Waste burning \n kg NO$_{x}$/ha")
axs[0, 1].boxplot(emiss_boxplot1[1], 0, '', labels = [''], showmeans=True)
axs[0, 1].set_title("Waste burning \n kg CH$_{4}$/ha")
axs[0, 2].boxplot(emiss_boxplot1[2], 0, '', labels = [''], showmeans=True)
axs[0, 2].set_title("Waste burning \n kg CO$_{2}$/ha")
axs[1, 0].boxplot(emiss_boxplot1[3], 0, '', labels = [''], showmeans=True)
axs[1, 0].set_title("Waste burning \n kg kg CO/ha")
axs[1, 1].boxplot(emiss_boxplot1[4], 0, '', labels = [''], showmeans=True)
axs[1, 1].set_title("Waste burning \n kg N$_{2}$O/ha")
axs[1, 2].boxplot(emiss_boxplot1[5], 0, '', labels = [''], showmeans=True)
axs[1, 2].set_title("Manure \n kg NH$_{3}$/ha")
axs[2, 0].boxplot(emiss_boxplot1[6], 0, '', labels = [''], showmeans=True)
axs[2, 0].set_title("Mineral fertiliser \n kg NH$_{3}$/ha") 
axs[2, 1].boxplot(emiss_boxplot1[7], 0, '', labels = [''], showmeans=True)
axs[2, 1].set_title("Struvite \n kg NH$_{3}$/ha") 
axs[2, 2].boxplot(emiss_boxplot1[8], 0, '', labels = [''], showmeans=True)
axs[2, 2].set_title("Compost \n kg NH$_{3}$/ha") 
axs[3, 0].boxplot(emiss_boxplot1[9], 0, '', labels = [''], showmeans=True)
axs[3, 0].set_title("Total \n kg NH$_{3}$/ha") 
axs[3, 1].boxplot(emiss_boxplot1[10], 0, '', labels = [''], showmeans=True)
axs[3, 1].set_title("Manure/Min fert \n kg NO$_{x}$/ha") 
axs[3, 2].boxplot(emiss_boxplot1[11], 0, '', labels = [''], showmeans=True)
axs[3, 2].set_title("Struvite \n kg NO$_{x}$/ha") 
axs[4, 0].boxplot(emiss_boxplot1[12], 0, '', labels = [''], showmeans=True)
axs[4, 0].set_title("Total \n kg NO$_{x}$/ha") 
axs[4, 1].boxplot(emiss_boxplot1[13], 0, '', labels = [''], showmeans=True)
axs[4, 1].set_title("Manure/Min fert/Residues \n kg NO$_{3}^-$/ha")
axs[4, 2].boxplot(emiss_boxplot1[14], 0, labels = [''], showmeans=True)
axs[4, 2].set_title("Struvite \n kg NO$_{3}^-$/ha")
axs[5, 0].boxplot(emiss_boxplot1[15], 0, '', labels = [''], showmeans=True)
axs[5, 0].set_title("Compost \n kg NO$_{3}^-$/ha") 
axs[5, 1].boxplot(emiss_boxplot1[16], 0, '', labels = [''], showmeans=True)
axs[5, 1].set_title("Total-Base leaching \n kg NO$_{3}^-$/ha") 
axs[5, 2].boxplot(emiss_boxplot1[17], 0, '', labels = [''], showmeans=True)
axs[5, 2].set_title("Direct(Manure, Min Fert, Res) \n kg N$_{2}$O/ha") 
axs[6, 0].boxplot(emiss_boxplot1[18], 0, '', labels = [''], showmeans=True)
axs[6, 0].set_title("Direct(Struvite) \n kg N$_{2}$O/ha")
axs[6, 1].boxplot(emiss_boxplot1[19], 0, '', labels = [''], showmeans=True)
axs[6, 1].set_title("Direct(Compost) \n kg N$_{2}$O/ha")
axs[6, 2].boxplot(emiss_boxplot1[20], 0, '', labels = [''], showmeans=True)
axs[6, 2].set_title("Direct(Total) \n kg N$_{2}$O/ha")
axs[7, 0].boxplot(emiss_boxplot1[21], 0, '', labels = [''], showmeans=True)
axs[7, 0].set_title("Indirect,Volatilization \n kg N$_{2}$O/ha")
axs[7, 1].boxplot(emiss_boxplot1[22], 0, '', labels = [''], showmeans=True)
axs[7, 1].set_title("Indirect,Leaching and runoff \n kg N$_{2}$O/ha")
axs[7, 2].boxplot(emiss_boxplot1[23], 0, '', labels = [''], showmeans=True)
axs[7, 2].set_title("Indirect,Total \n kg N$_{2}$O/ha")
axs[8, 0].boxplot(emiss_boxplot1[24], 0, '', labels = [''], showmeans=True)
axs[8, 0].set_title("Total \n kg N$_{2}$O/ha")
axs[8, 1].boxplot(emiss_boxplot1[25], 0, '', labels = [''], showmeans=True)
axs[8, 1].set_title("Urea \n kg CO$_{2}$/ha")
axs[8, 2].boxplot(emiss_boxplot1[26], 0, '', labels = [''], showmeans=True)
axs[8, 2].set_title("Total \n kg CO$_{2}$/ha")
axs[9, 0].boxplot(emiss_boxplot1[27], 0, '', labels = [''], showmeans=True)
axs[9, 0].set_title("Manure/Min fert runoff \n kg P/ha")
axs[9, 1].boxplot(emiss_boxplot1[28], 0, '', labels = [''], showmeans=True)
axs[9, 1].set_title("Struvite runoff \n kg P/ha")
axs[9, 2].boxplot(emiss_boxplot1[29], 0, '', labels = [''], showmeans=True)
axs[9, 2].set_title("Total-Base runoff \n kg P/ha")
fig.tight_layout()
#fig.savefig( 'output/' + scen + '/emissions/Direct_Emissions' + '.png')